In [4]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import random
from prompt_templates import create_prompt_subtitles
from transformers import MT5ForConditionalGeneration
import argparse
from datetime import datetime
from models import get_model
from dataloader import get_loader
import torch
from torch import nn
from evaluate import evaluate
import copy
import wandb
from tqdm import tqdm
import os
import json
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer
from scheduler import CosineWarmupScheduler
from utils import compute_rank_loss
from torch.cuda.amp import GradScaler


In [3]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.2 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 10.2 MB/s eta 0:00:0000:0100:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 9.5 MB/s eta 0:00:00:00:0100:01


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/colin/miniconda3/envs/recsys/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [14]:
model.base_model.load_state_dict(torch.load('checkpoints/model_0.0001.pth',map_location=torch.device('cpu')))

<All keys matched successfully>

In [23]:
class b:
    dataset = "demo"
    batch_size = 3
    num_workers = 4
    titles = True
    T=4
    datafraction=0.001

b = b()

In [24]:
data_loader = get_loader(b, 'train', tokenizer, debug=True)

In [25]:
data_loader2 = get_loader(b, 'train', tokenizer, debug=False)


In [26]:
out1= next(iter(data_loader2))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/colin/miniconda3/envs/recsys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/colin/

In [31]:
out1.keys()

dict_keys(['pos_input_ids', 'pos_attention_mask', 'neg_input_ids', 'neg_attention_mask', 'pos_labels', 'neg_labels', 'decoder_start'])

In [27]:
out1['pos_input_ids'].shape, out1['pos_attention_mask'].shape, out1['neg_input_ids'].shape, out1['pos_labels'].shape, out1['neg_attention_mask'].shape

(torch.Size([3, 113]),
 torch.Size([3, 113]),
 torch.Size([3, 110]),
 torch.Size([3, 2]),
 torch.Size([3, 110]))

In [36]:
out1["decoder_start"].shape

torch.Size([3, 5])

In [28]:
for i in range(3):
    print(tokenizer.decode(out1['pos_input_ids'][i].tolist()))
    print(tokenizer.decode(out1['neg_input_ids'][i].tolist()))  

En bruger har for nylig læst artikler: Holger Runes showkamp i Royal Arena er aflyst, Putins 'vasketrick': Nu reagerer de, Oppositionsleder i nødråb: - Han sælger vores land, Mikkel Kessler sælger huset, vil brugeren læse artiklen Netbank ramt af nedbrud? (ja/nej)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
En bruger har for nylig læst artikler: Holger Runes showkamp i Royal Arena er aflyst, Putins 'vasketrick': Nu reagerer de, Oppositionsleder i nødråb: - Han sælger vores land, Mikkel Kessler sælger huset, vil brugeren læse artiklen Ukraine klar til modangreb: Her kan de presse Putin? (ja/nej)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
En bruger har for nylig læst artikler: Midt i skole-fejring: Rykker ud til knivstikkeri, Sender stikpille til dommer: - Han læser aftrykkene forkert, Knivstikkeri i Fælledparken: En anholdt og sigtet, Total ydmygelse: City s

In [ ]:
tokenizer.encode("ja / nej"), tokenizer.encode("ja"), tokenizer.encode("nej")

([432, 259, 275, 3810, 1], [432, 1], [3810, 1])

In [40]:
pos_outputs = model(
    input_ids=out1["pos_input_ids"], 
    attention_mask=out1["pos_attention_mask"],
    decoder_input_ids=out1["decoder_start"][:,],
)
neg_outputs = model(
    input_ids=out1["neg_input_ids"], 
    attention_mask=out1["neg_attention_mask"],
    decoder_input_ids=out1["decoder_start"],
)


In [52]:
ja = tokenizer.encode('ja')[0]
nej = tokenizer.encode('nej')[0]

other_logit_idxs = [i for i in range(pos_outputs.logits.shape[2])]
other_logit_idxs.remove(ja)
other_logit_idxs.remove(nej)

print('== pos examples ==')
print('ja:',pos_outputs.logits[:,0,ja])
print('nej:',pos_outputs.logits[:,0,nej])
other_logits_pos = pos_outputs.logits[:,0,other_logit_idxs]
print('all other tokens (mean):',torch.mean(other_logits_pos,dim=-1))
print('all other tokens (std):',torch.std(other_logits_pos,dim=-1))



print('== pos examples ==')
print('ja:',neg_outputs.logits[:,0,ja])
print('nej:',neg_outputs.logits[:,0,nej])
other_logits_neg = neg_outputs.logits[:,0,other_logit_idxs]
print('all other tokens (mean):',torch.mean(other_logits_neg,dim=-1))
print('all other tokens (std):',torch.std(other_logits_neg,dim=-1))

== pos examples ==
ja: tensor([4.3720, 4.0491, 4.2663], grad_fn=<SelectBackward0>)
nej: tensor([5.5062, 5.7165, 5.4130], grad_fn=<SelectBackward0>)
all other tokens (mean): tensor([-22.7337, -22.8223, -22.8444], grad_fn=<MeanBackward1>)
all other tokens (std): tensor([0.4487, 0.4915, 0.4547], grad_fn=<StdBackward0>)
== pos examples ==
ja: tensor([5.0834, 4.0282, 4.1644], grad_fn=<SelectBackward0>)
nej: tensor([4.3363, 5.6186, 5.9130], grad_fn=<SelectBackward0>)
all other tokens (mean): tensor([-22.6058, -22.9179, -22.8858], grad_fn=<MeanBackward1>)
all other tokens (std): tensor([0.4334, 0.4995, 0.4922], grad_fn=<StdBackward0>)


In [54]:
pos_outputs.logits.shape

torch.Size([3, 2, 250112])

In [30]:
out1.keys()

dict_keys(['pos_input_ids', 'pos_attention_mask', 'neg_input_ids', 'neg_attention_mask', 'pos_labels', 'neg_labels', 'empt'])

In [43]:
print(tokenizer.decode(out1['pos_input_ids'][0].tolist()))
print(tokenizer.decode(out1['neg_input_ids'][0].tolist()))

En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen - Hvis nogen tror, at vi lægger os fladt ned mod FC Nordsjælland, så må det være personer, der ikke er involveret i elitesport, siger en vred Carsten V. Jensen? (ja/nej)</s>
En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. If

"En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen Aktien er faldet med knap 30 procent onsdag? (ja/nej)</s>"

In [37]:
tokenizer.encode("hello")

[124393, 1]

In [27]:
out1, out2 = next(iter(data_loader2)), next(next(iter(data_loader2)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is dep

TypeError: 'dict' object is not an iterator